## Initial Model
I'm going to try to create a regression model to predict voter turnout based on variables like land ownership, party, age and gender. In our initial EDA we saw that the state of Michigan had a significant different in voter turnout between political parties in the 2016 eleciton so the first model will be just using Michigan data.

### Loaidng the Data

In [2]:
gcs_path = 'gs://pstat135-voter-file/VM2Uniform'

mi =  spark.read.parquet(gcs_path + "/" + "VM2Uniform--MI--2021-01-30")


In [3]:
# mi.printSchema()


In [32]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pandas as pd

In [5]:
mod_df = mi.select("Residence_Addresses_Property_LandSq_Footage",
        "Parties_Description",
        "Voters_Age",
        "CommercialData_Education",
        "CommercialData_EstimatedHHIncomeAmount",
        "Voters_Age",
        "General_2020")

In [6]:
total = mod_df.count()

In [7]:
from pyspark.sql.functions import col,isnan, when, count
missing_df = mod_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in mod_df.columns]
   )



In [8]:
missing_df_pd = missing_df.toPandas().transpose()
missing_df_pd["PercentMissing"] = missing_df_pd[0] / total
missing_df_pd.columns = ["Total Missing", "Percent Missing"]
missing_df_pd

,Total Missing,Percent Missing
Residence_Addresses_Property_LandSq_Footage,1604310,0.211270
Parties_Description,0,0.000000
Voters_Age,12456,0.001640
CommercialData_Education,3089297,0.406826
CommercialData_EstimatedHHIncomeAmount,150772,0.019855
Voters_Age,12456,0.001640
General_2020,2197299,0.289360


In [9]:
mod_df = mod_df.withColumn("label", when(mod_df.General_2020 == "Y", 1)
                                            .otherwise(0))
mod_df = mod_df.drop("General_2020")
mod_df = mod_df.withColumn("Residence_Addresses_Property_LandSq_Footage",
                           mod_df["Residence_Addresses_Property_LandSq_Footage"].cast(IntegerType()))

mod_df = mod_df.withColumn("Age",
                           mod_df["Voters_Age"].cast(IntegerType()))
mod_df = mod_df.drop("Voters_Age")

mod_df = mod_df.withColumn("Income", regexp_replace("CommercialData_EstimatedHHIncomeAmount", "\$", "").cast(IntegerType()))
mod_df = mod_df.drop("CommercialData_EstimatedHHIncomeAmount")

mod_df.show(5)
mod_df.count()

+-------------------------------------------+-------------------+------------------------+-----+---+------+
|Residence_Addresses_Property_LandSq_Footage|Parties_Description|CommercialData_Education|label|Age|Income|
+-------------------------------------------+-------------------+------------------------+-----+---+------+
|                                      26000|         Democratic|    HS Diploma - Extr...|    1| 58|109000|
|                                      17000|         Republican|    Some College - Li...|    1| 29| 65000|
|                                      17000|         Republican|    Some College - Li...|    1| 57| 65000|
|                                      17000|         Republican|    Some College - Li...|    1| 56| 65000|
|                                      36000|         Republican|    Bach Degree - Likely|    1| 62| 71000|
+-------------------------------------------+-------------------+------------------------+-----+---+------+
only showing top 5 rows



7593651

In [10]:
mod_df = mod_df.dropna()
mod_df.count()

3816666

### Preparing data for Model
Now I need to one hot encode the `Parties_Description` and `CommercialData_Education` columns and create a column with all the features.

In [11]:
# Create an indexer
indexer = StringIndexer(inputCol="Parties_Description", outputCol='Parties_Description_idx')
edu_indexer = StringIndexer(inputCol="CommercialData_Education", outputCol='CommercialData_Education_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(mod_df)
edu_indexer_model = edu_indexer.fit(mod_df)

# Indexer creates a new column with numeric index values
df_indexed = indexer_model.transform(mod_df)
df_indexed = edu_indexer_model.transform(df_indexed)
onehot = OneHotEncoder(inputCols=["Parties_Description_idx", "CommercialData_Education_idx"],
                       outputCols=["party_encoded", "education_encoded"])
onehot = onehot.fit(df_indexed)

### CommercialData_Education ###

df_onehot = onehot.transform(df_indexed)

In [36]:
df_onehot.select("CommercialData_Education", "CommercialData_Education_idx", "education_encoded").show(10)

+------------------------+----------------------------+-----------------+
|CommercialData_Education|CommercialData_Education_idx|education_encoded|
+------------------------+----------------------------+-----------------+
|    HS Diploma - Extr...|                         0.0|   (10,[0],[1.0])|
|    Some College - Li...|                         1.0|   (10,[1],[1.0])|
|    Some College - Li...|                         1.0|   (10,[1],[1.0])|
|    Some College - Li...|                         1.0|   (10,[1],[1.0])|
|    Bach Degree - Likely|                         4.0|   (10,[4],[1.0])|
|     HS Diploma - Likely|                         3.0|   (10,[3],[1.0])|
|    HS Diploma - Extr...|                         0.0|   (10,[0],[1.0])|
|    Some College - Li...|                         1.0|   (10,[1],[1.0])|
|    Bach Degree - Ext...|                         2.0|   (10,[2],[1.0])|
|    Some College - Li...|                         1.0|   (10,[1],[1.0])|
+------------------------+------------

In [56]:
edu_codes = df_onehot.groupBy("CommercialData_Education_idx", "CommercialData_Education").count().sort("CommercialData_Education_idx")

edu_codes.show()

+----------------------------+------------------------+------+
|CommercialData_Education_idx|CommercialData_Education| count|
+----------------------------+------------------------+------+
|                         0.0|    HS Diploma - Extr...|781282|
|                         1.0|    Some College - Li...|696285|
|                         2.0|    Bach Degree - Ext...|694970|
|                         3.0|     HS Diploma - Likely|355231|
|                         4.0|    Bach Degree - Likely|322629|
|                         5.0|    Grad Degree - Ext...|321905|
|                         6.0|    Grad Degree - Likely|248123|
|                         7.0|    Some College -Ext...|245145|
|                         8.0|    Less than HS Dipl...|142493|
|                         9.0|    Vocational Techni...|  7108|
|                        10.0|    Less than HS Dipl...|  1495|
+----------------------------+------------------------+------+



In [61]:
edu_codes.select("CommercialData_Education")[]

Column<'CommercialData_Education'>

In [37]:
df_onehot.select("Parties_Description", "Parties_Description_idx", "party_encoded").show(10)

+-------------------+-----------------------+-------------+
|Parties_Description|Parties_Description_idx|party_encoded|
+-------------------+-----------------------+-------------+
|         Democratic|                    0.0|(2,[0],[1.0])|
|         Republican|                    1.0|(2,[1],[1.0])|
|         Republican|                    1.0|(2,[1],[1.0])|
|         Republican|                    1.0|(2,[1],[1.0])|
|         Republican|                    1.0|(2,[1],[1.0])|
|         Republican|                    1.0|(2,[1],[1.0])|
|         Republican|                    1.0|(2,[1],[1.0])|
|         Democratic|                    0.0|(2,[0],[1.0])|
|         Democratic|                    0.0|(2,[0],[1.0])|
|       Non-Partisan|                    2.0|    (2,[],[])|
+-------------------+-----------------------+-------------+
only showing top 10 rows



In [53]:
# Create an assembler object
assembler = VectorAssembler(inputCols=[
    "Residence_Addresses_Property_LandSq_Footage",
    "Age",
    "Income",
    "party_encoded",
    "education_encoded"
], outputCol='features')

# Consolidate predictor columns
df_assembled = assembler.transform(df_onehot)

# Check the resulting column
df_assembled.select('features', 'label').show(5, truncate=False)

+------------------------------------------------+-----+
|features                                        |label|
+------------------------------------------------+-----+
|(15,[0,1,2,3,5],[26000.0,58.0,109000.0,1.0,1.0])|1    |
|(15,[0,1,2,4,6],[17000.0,29.0,65000.0,1.0,1.0]) |1    |
|(15,[0,1,2,4,6],[17000.0,57.0,65000.0,1.0,1.0]) |1    |
|(15,[0,1,2,4,6],[17000.0,56.0,65000.0,1.0,1.0]) |1    |
|(15,[0,1,2,4,9],[36000.0,62.0,71000.0,1.0,1.0]) |1    |
+------------------------------------------------+-----+
only showing top 5 rows



#### Train Test Split

In [54]:
df_train, df_test = df_assembled.randomSplit([0.8, 0.2], seed=43)

### Logistic Regression Model

In [66]:
logistic = LogisticRegression().fit(df_train)

In [67]:
prediction = logistic.transform(df_test)
prediction.groupBy("label", "prediction").count().show()

+-----+----------+------+
|label|prediction| count|
+-----+----------+------+
|    1|       0.0|  3156|
|    0|       0.0|  2368|
|    1|       1.0|627049|
|    0|       1.0|130000|
+-----+----------+------+



In [68]:
summary = logistic.summary
summary

In [46]:
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName='areaUnderROC')

evaluator.evaluate(prediction)

0.5002808361032538

In [47]:
evaluator.explainParams()

'labelCol: label column name. (default: label, current: label)\nmetricName: metric name in evaluation (areaUnderROC|areaUnderPR) (default: areaUnderROC, current: areaUnderROC)\nnumBins: Number of bins to down-sample the curves (ROC curve, PR curve) in area computation. If 0, no down-sampling will occur. Must be >= 0. (default: 1000)\nrawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction, current: prediction)\nweightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)'

In [48]:
evaluator.getWeightCol()

KeyError: Param(parent='BinaryClassificationEvaluator_383c69442ded', name='weightCol', doc='weight column name. If this is not set or empty, we treat all instance weights as 1.0.')

In [ ]:
pd.DataFrame({'coefficients':logistic.coefficients, 'feature':list(pd.DataFrame(df_train.schema["features"].metadata["ml_attr"]["attrs"]['numeric']).sort_values('idx')['name'])})




In [49]:
def ExtractFeatureCoeficient(model, dataset, excludedCols = None):
    test = model.transform(dataset)
    weights = model.coefficients
    print('This is model weights: \n', weights)
    weights = [(float(w),) for w in weights]  # convert numpy type to float, and to tuple
    if excludedCols == None:
        feature_col = [f for f in test.schema.names if f not in ['y', 'classWeights', 'features', 'label', 'rawPrediction', 'probability', 'prediction']]
    else:
        feature_col = [f for f in test.schema.names if f not in excludedCols]
    if len(weights) == len(feature_col):
        weightsDF = sqlContext.createDataFrame(zip(weights, feature_col), schema= ["Coeficients", "FeatureName"])
    else:
        print('Coeficients are not matching with remaining Fetures in the model, please check field lists with model.transform(dataset).schema.names')
    
    return weightsDF

In [50]:
results = ExtractFeatureCoeficient(logistic, df_train)

This is model weights: 
 [1.089365254494632e-09,0.029973694111211854,8.297718297099569e-06]
Coeficients are not matching with remaining Fetures in the model, please check field lists with model.transform(dataset).schema.names


UnboundLocalError: local variable 'weightsDF' referenced before assignment

In [52]:
logistic.transform(df_train).schema.names

['Residence_Addresses_Property_LandSq_Footage',
 'Parties_Description',
 'CommercialData_Education',
 'label',
 'Age',
 'Income',
 'Parties_Description_idx',
 'CommercialData_Education_idx',
 'party_encoded',
 'education_encoded',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [63]:
edu_code0 = edu_codes.filter(edu_codes.CommercialData_Education_idx == 0).head()[1]
edu_code1 = edu_codes.filter(edu_codes.CommercialData_Education_idx == 1).head()[1]
edu_code2 = edu_codes.filter(edu_codes.CommercialData_Education_idx == 2).head()[1]
edu_code3 = edu_codes.filter(edu_codes.CommercialData_Education_idx == 3).head()[1]
edu_code4 = edu_codes.filter(edu_codes.CommercialData_Education_idx == 4).head()[1]
edu_code5 = edu_codes.filter(edu_codes.CommercialData_Education_idx == 5).head()[1]
edu_code6 = edu_codes.filter(edu_codes.CommercialData_Education_idx == 6).head()[1]
edu_code7 = edu_codes.filter(edu_codes.CommercialData_Education_idx == 7).head()[1]
edu_code8 = edu_codes.filter(edu_codes.CommercialData_Education_idx == 8).head()[1]
edu_code9 = edu_codes.filter(edu_codes.CommercialData_Education_idx == 9).head()[1]
edu_code10 = edu_codes.filter(edu_codes.CommercialData_Education_idx == 10).head()[1]

In [69]:
coef_order = ["Residence_Addresses_Property_LandSq_Footage",
             "Age",
             "Income",
             "Republican",
             "Non-partisan",
              edu_code1,
              edu_code2,
              edu_code3,
              edu_code4,
              edu_code5,
              edu_code6,
              edu_code7,
              edu_code8,
              edu_code9,
              edu_code10
             ]

coef_df = pd.DataFrame(list(zip(coef_order, logistic.coefficients)),
                      columns =['val', 'coef'])

In [70]:
coef_df

,val,coef
0,Residence_Addresses_Property_LandSq_Footage,-2.477099e-09
1,Age,2.479320e-02
2,Income,7.688554e-06
3,Republican,8.001599e-01
4,Non-partisan,1.091068e+00
5,Some College - Likely,6.780110e-01
6,Bach Degree - Extremely Likely,8.776986e-01
7,HS Diploma - Likely,1.086990e+00
8,Bach Degree - Likely,6.645319e-01
9,Grad Degree - Extremely Likely,1.036221e+00
